In [49]:
%%bash
set -e
rm -rf /content/slm-hosting-playbook
git clone https://github.com/bh3r1th/slm-hosting-playbook.git /content/slm-hosting-playbook


Cloning into '/content/slm-hosting-playbook'...


In [2]:
%%bash
set -e
python -m pip -q install -U pip
python -m pip -q install "vllm" "jedi>=0.16"
python -m pip -q install "requests==2.32.4"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 79.1 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [4]:
%%bash
set -e
export VLLM_HOST=0.0.0.0
export VLLM_PORT_BASE=8000
export VLLM_PORT_FT=8001
export BASE_MODEL_ID="google/gemma-3-1b-it"
export FT_SERVED_MODEL_NAME="ft"
export GPU_MEMORY_UTILIZATION=0.90
export MAX_MODEL_LEN=2048
export TENSOR_PARALLEL_SIZE=1

export ADAPTER_PATH="/content/drive/MyDrive/slm-hosting-playbook-artifacts/live_backup/gemma-3-1b-it-lora-20260114-034307"

# sanity: adapter files must exist
ls -lah "$ADAPTER_PATH"/adapter_config.json "$ADAPTER_PATH"/adapter_model.safetensors


-rw------- 1 root root 1.1K Jan 14 09:19 /content/drive/MyDrive/slm-hosting-playbook-artifacts/live_backup/gemma-3-1b-it-lora-20260114-034307/adapter_config.json
-rw------- 1 root root  25M Jan 14 09:19 /content/drive/MyDrive/slm-hosting-playbook-artifacts/live_backup/gemma-3-1b-it-lora-20260114-034307/adapter_model.safetensors


In [8]:
%%bash
set -e

# show what the wrapper script is
cd /content/slm-hosting-playbook/gemma-slm-hosting
sed -n '1,200p' scripts/start_base_vllm.sh

# show last log
echo "==== last 400 lines of base log ===="
tail -n 400 /content/vllm_base.log || true


#!/usr/bin/env bash
# This script starts vLLM in OpenAI-compatible mode (v1/* endpoints) for local testing.
set -euo pipefail

script_dir="$(cd "$(dirname "${BASH_SOURCE[0]}")" && pwd)"

if [ -f "$script_dir/.env" ]; then
  declare -A env_overrides=()
  while IFS= read -r line; do
    case "$line" in
      ''|\#*) continue ;;
    esac
    if [[ "$line" =~ ^[[:space:]]*(export[[:space:]]+)?([A-Za-z_][A-Za-z0-9_]*)= ]]; then
      key="${BASH_REMATCH[2]}"
      if printenv "$key" >/dev/null 2>&1; then
        env_overrides["$key"]="$(printenv "$key")"
      fi
    fi
  done < "$script_dir/.env"
  set -a
  # shellcheck disable=SC1090
  . "$script_dir/.env"
  set +a
  for key in "${!env_overrides[@]}"; do
    export "$key=${env_overrides[$key]}"
  done
fi

eval "$(python "$script_dir/read_pointer.py")"

: "${GPU_MEMORY_UTILIZATION:=0.90}"
: "${MAX_MODEL_LEN:=2048}"
: "${TENSOR_PARALLEL_SIZE:=1}"
: "${DTYPE:=auto}"

if [ -z "${BASE_MODEL_ID:-}" ]; then
  echo "Missing required BASE_MODEL_ID

In [23]:
%%bash
set -e
cd /content/slm-hosting-playbook/gemma-slm-hosting

pkill -f "vllm.entrypoints.openai.api_server" || true

python -m vllm.entrypoints.openai.api_server \
  --host 0.0.0.0 \
  --port 8000 \
  --model google/gemma-3-1b-it \
  --gpu-memory-utilization 0.90 \
  --max-model-len 2048 \
  --tensor-parallel-size 1 \
  > /content/vllm_base.log 2>&1 &

echo $! > /content/vllm_base.pid
echo "vLLM pid=$(cat /content/vllm_base.pid)"
sleep 3
tail -n 120 /content/vllm_base.log || true


vLLM pid=11161


In [24]:
%%bash
set -e
ps -p $(cat /content/vllm_base.pid) -o pid,cmd || true
ss -ltnp | grep ':8000' || true
tail -n 200 /content/vllm_base.log || true


    PID CMD
  11161 python3 -m vllm.entrypoints.openai.api_server --host 0.0.0.0 --port 8000 --model google/gemma-3-1b-it --gpu-memory-utilization 0.90 --max-model-len 2048 --tensor-parallel-size 1


In [26]:
%%bash
set -e
ps -p $(cat /content/vllm_base.pid) -o pid,cmd || true
ss -ltnp | grep ':8000' || true
tail -n 200 /content/vllm_base.log || true


    PID CMD
  11161 python3 -m vllm.entrypoints.openai.api_server --host 0.0.0.0 --port 8000 --model google/gemma-3-1b-it --gpu-memory-utilization 0.90 --max-model-len 2048 --tensor-parallel-size 1
2026-01-16 03:59:27.338135: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-16 03:59:27.355577: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768535967.376717   11161 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768535967.383219   11161 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory 

In [40]:
%%bash
set -e
cd /content/slm-hosting-playbook/gemma-slm-hosting

export VLLM_HOST=0.0.0.0
export VLLM_PORT_BASE=8000
export BASE_MODEL_ID="google/gemma-3-1b-it"

# lower this a bit to be safe
export GPU_MEMORY_UTILIZATION=0.60
export MAX_MODEL_LEN=2048
export TENSOR_PARALLEL_SIZE=1
export DTYPE=auto

nohup bash scripts/start_base_vllm.sh > /content/vllm_base.log 2>&1 & echo $! > /content/vllm_base.pid

# wait until ready
for i in $(seq 1 180); do
  curl -sf http://127.0.0.1:8000/v1/models >/dev/null && echo "BASE READY" && break
  sleep 2
done

curl -s http://127.0.0.1:8000/v1/models | head -c 300; echo


BASE READY
{"object":"list","data":[{"id":"google/gemma-3-1b-it","object":"model","created":1768537607,"owned_by":"vllm","root":"google/gemma-3-1b-it","parent":null,"max_model_len":2048,"permission":[{"id":"modelperm-89550fc31efe77fe","object":"model_permission","created":1768537607,"allow_create_engine":false


In [44]:
%%bash
set -e
curl -s http://127.0.0.1:8000/v1/models | head -c 300; echo


{"object":"list","data":[{"id":"google/gemma-3-1b-it","object":"model","created":1768538939,"owned_by":"vllm","root":"google/gemma-3-1b-it","parent":null,"max_model_len":2048,"permission":[{"id":"modelperm-b528e7d8520cd7d1","object":"model_permission","created":1768538939,"allow_create_engine":false


In [47]:
%%bash
set -e
curl -sf http://127.0.0.1:8000/v1/models >/dev/null
python - <<'PY'
import requests
url="http://127.0.0.1:8000/v1/chat/completions"
payload={
  "model":"google/gemma-3-1b-it",
  "messages":[{"role":"user","content":"Say 'OK' and nothing else."}],
  "temperature":0.0,
  "max_tokens":16
}
r=requests.post(url, json=payload, timeout=120)
r.raise_for_status()
print("BASE_SMOKE_OK:", r.json()["choices"][0]["message"]["content"])
PY


BASE_SMOKE_OK: OK.


In [50]:
%%bash
set -e
cd /content/slm-hosting-playbook/gemma-slm-hosting
python scripts/smoke_test.py --mode base


BASE: Hello there!


In [51]:
%%bash
set -e
ss -ltnp | egrep ':8000|:8001' || true
ps aux | grep -E "vllm.*api_server" | grep -v grep || true


LISTEN 0      2048         0.0.0.0:8000       0.0.0.0:*    users:(("python3",pid=20204,fd=33))         
root       20204  0.9  2.2 14227688 1929848 ?    Sl   04:25   0:24 python3 -m vllm.entrypoints.openai.api_server --host 0.0.0.0 --port 8000 --model google/gemma-3-1b-it --gpu-memory-utilization 0.60 --max-model-len 2048 --tensor-parallel-size 1 --dtype auto


In [52]:
%%bash
set -e
cd /content/slm-hosting-playbook/gemma-slm-hosting
python scripts/smoke_test.py --mode base
echo "SMOKE_TEST_BASE_ONLY_OK"


BASE: Hello there!
SMOKE_TEST_BASE_ONLY_OK


In [54]:
%%bash
set -e
pkill -f "vllm.entrypoints.openai.api_server" || true
pkill -f "vllm.*api_server" || true
sleep 2
nvidia-smi


Fri Jan 16 05:24:35 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P0             69W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [55]:
%%bash
set -e
cd /content/slm-hosting-playbook/gemma-slm-hosting

export VLLM_HOST=0.0.0.0
export VLLM_PORT_FT=8001
export BASE_MODEL_ID="google/gemma-3-1b-it"
export ADAPTER_PATH="/content/drive/MyDrive/slm-hosting-playbook-artifacts/live_backup/gemma-3-1b-it-lora-20260114-034307"

# hard anti-OOM settings
export GPU_MEMORY_UTILIZATION=0.45
export MAX_MODEL_LEN=2048
export MAX_NUM_SEQS=8
export ENFORCE_EAGER=1

nohup bash scripts/start_ft_vllm.sh > /content/vllm_ft.log 2>&1 & echo $! > /content/vllm_ft.pid

# wait until ready
for i in $(seq 1 180); do
  curl -sf http://127.0.0.1:8001/v1/models >/dev/null && echo "FT READY" && break
  sleep 2
done

curl -s http://127.0.0.1:8001/v1/models | head -c 300; echo
tail -n 80 /content/vllm_ft.log || true


FT READY
{"object":"list","data":[{"id":"ft","object":"model","created":1768541183,"owned_by":"vllm","root":"google/gemma-3-1b-it","parent":null,"max_model_len":2048,"permission":[{"id":"modelperm-b20757271758221a","object":"model_permission","created":1768541183,"allow_create_engine":false,"allow_sampling":
W0000 00:00:1768541125.014350   36376 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768541125.014378   36376 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768541125.014392   36376 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768541125.014394   36376 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more t

In [57]:
%%bash
set -e
cd /content/slm-hosting-playbook/gemma-slm-hosting
python scripts/smoke_test.py --mode ft
echo "SMOKE_TEST_FT_ONLY_OK"


FT: Hello! 😊
SMOKE_TEST_FT_ONLY_OK
